# Experimental Setup for Building Classifiers:

## Egyptian Wikipedia Dataset Encoding Using CAMeLBERT:

### Encoding Procedure Using "[CAMeLBERT-Mix POS-EGY Model](https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-mix-pos-egy)":

In [3]:
import torch
import numpy as np
from transformers import AutoModel
from transformers import BertTokenizer
from IPython.display import clear_output

articles = 0
pages_embeddings = {}

def encode_page_text(page_text, dataframe_name): 
    global articles
    articles += 1
    
    idx = np.where(dataframe_name["page_text_cleaned"] == page_text)[0][0]
    page_title = dataframe_name['page_title'][idx]
    
    tokenizer = BertTokenizer.from_pretrained('CAMeL-Lab/bert-base-arabic-camelbert-mix-pos-egy')
    model = AutoModel.from_pretrained('CAMeL-Lab/bert-base-arabic-camelbert-mix-pos-egy')

    tokenized_page_text  = tokenizer(page_text, return_tensors='pt', max_length=512, truncation=True)
    encoded_page_text = model(**tokenized_page_text)[0][0][0].tolist()

    pages_embeddings[page_title] = encoded_page_text

    clear_output(wait=True)
    print(f"# Texts of {articles} pages have been encoded.", flush=True)
    
    return encoded_page_text

In [2]:
import pandas as pd, logging, warnings
from transformers import logging as tlogging

tlogging.set_verbosity_warning()
logging.disable(logging.WARNING)

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

df = pd.read_csv('arzwiki-20240101-all-20k.csv')
df.head()

,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text,page_text_cleaned,label
0,كريستينا نوير,27,6,1685,460,65,كريستينا نوير christina noir يوليه ممثلة بورنو...,كريستينا نوير يوليه ممثلة بورنوجرافية امريكية ...,human-generated
1,ارنست هيفليجر,2,2,1060,381,67,ارنست هيفليجر كان معلم موسيقى مغنى اوبرا مغنى ...,ارنست هيفليجر كان معلم موسيقى مغنى اوبرا مغنى ...,template-translated
2,ناديه مصطفى,19,8,1841,637,113,نادية مصطفى اكتوبر مغنية مصرية بدأت مشوارها ال...,نادية مصطفى اكتوبر مغنية مصرية بدأت مشوارها ال...,human-generated
3,جامعات عمان,10,6,1810,767,113,الجامعات ومؤسسات التعليم العالي فى سلطنه عمان ...,الجامعات ومؤسسات التعليم العالي فى سلطنه عمان ...,human-generated
4,الحرب السكندريه,51,16,12077,6192,1083,يوليوس قيصر خاض حرب اسكندريه الحرب السكندريه ح...,يوليوس قيصر خاض حرب اسكندريه الحرب السكندريه ح...,human-generated


In [4]:
df['page_text_encoded_camelbert'] = df['page_text_cleaned'].apply(lambda page_text: encode_page_text(page_text, df))
df.head()

# Texts of 20000 pages have been encoded.


,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text,page_text_cleaned,label,page_text_encoded_camelbert
0,كريستينا نوير,27,6,1685,460,65,كريستينا نوير christina noir يوليه ممثلة بورنو...,كريستينا نوير يوليه ممثلة بورنوجرافية امريكية ...,human-generated,"[0.9345402717590332, -0.23156806826591492, 1.0..."
1,ارنست هيفليجر,2,2,1060,381,67,ارنست هيفليجر كان معلم موسيقى مغنى اوبرا مغنى ...,ارنست هيفليجر كان معلم موسيقى مغنى اوبرا مغنى ...,template-translated,"[1.1935014724731445, -0.4615197479724884, 0.74..."
2,ناديه مصطفى,19,8,1841,637,113,نادية مصطفى اكتوبر مغنية مصرية بدأت مشوارها ال...,نادية مصطفى اكتوبر مغنية مصرية بدأت مشوارها ال...,human-generated,"[1.0213305950164795, -0.9650395512580872, 0.87..."
3,جامعات عمان,10,6,1810,767,113,الجامعات ومؤسسات التعليم العالي فى سلطنه عمان ...,الجامعات ومؤسسات التعليم العالي فى سلطنه عمان ...,human-generated,"[0.8735443353652954, -1.340850591659546, 0.825..."
4,الحرب السكندريه,51,16,12077,6192,1083,يوليوس قيصر خاض حرب اسكندريه الحرب السكندريه ح...,يوليوس قيصر خاض حرب اسكندريه الحرب السكندريه ح...,human-generated,"[1.1119786500930786, -0.9848897457122803, 0.63..."


### Exporting and Importing Word Embeddings:

In [5]:
import json
with open('arzwiki-20240101-all-20k-camelbert-embeddings.json', 'w') as json_file:
    json.dump(pages_embeddings, json_file)

In [6]:
import json
with open('arzwiki-20240101-all-20k-camelbert-embeddings.json') as json_file:
    fetched_pages_embeddings = json.load(json_file)

In [7]:
def fetch_embeddings(page_text, dataframe_name, embeddings_file):
    idx = np.where(dataframe_name["page_text_cleaned"] == page_text)[0][0]
    page_title = dataframe_name['page_title'][idx]
    return embeddings_file[page_title]

In [8]:
df['fetched_page_text_camelbert_embeddings'] = df['page_text_cleaned'].apply(lambda page_text: fetch_embeddings(page_text, df, fetched_pages_embeddings))
df.head()

,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text,page_text_cleaned,label,page_text_encoded_camelbert,fetched_page_text_camelbert_embeddings
0,كريستينا نوير,27,6,1685,460,65,كريستينا نوير christina noir يوليه ممثلة بورنو...,كريستينا نوير يوليه ممثلة بورنوجرافية امريكية ...,human-generated,"[0.9345402717590332, -0.23156806826591492, 1.0...","[0.9345402717590332, -0.23156806826591492, 1.0..."
1,ارنست هيفليجر,2,2,1060,381,67,ارنست هيفليجر كان معلم موسيقى مغنى اوبرا مغنى ...,ارنست هيفليجر كان معلم موسيقى مغنى اوبرا مغنى ...,template-translated,"[1.1935014724731445, -0.4615197479724884, 0.74...","[1.1935014724731445, -0.4615197479724884, 0.74..."
2,ناديه مصطفى,19,8,1841,637,113,نادية مصطفى اكتوبر مغنية مصرية بدأت مشوارها ال...,نادية مصطفى اكتوبر مغنية مصرية بدأت مشوارها ال...,human-generated,"[1.0213305950164795, -0.9650395512580872, 0.87...","[1.0213305950164795, -0.9650395512580872, 0.87..."
3,جامعات عمان,10,6,1810,767,113,الجامعات ومؤسسات التعليم العالي فى سلطنه عمان ...,الجامعات ومؤسسات التعليم العالي فى سلطنه عمان ...,human-generated,"[0.8735443353652954, -1.340850591659546, 0.825...","[0.8735443353652954, -1.340850591659546, 0.825..."
4,الحرب السكندريه,51,16,12077,6192,1083,يوليوس قيصر خاض حرب اسكندريه الحرب السكندريه ح...,يوليوس قيصر خاض حرب اسكندريه الحرب السكندريه ح...,human-generated,"[1.1119786500930786, -0.9848897457122803, 0.63...","[1.1119786500930786, -0.9848897457122803, 0.63..."
